In [3]:
#Implementation of S-DES
# Initial and final permutations
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_INV = [4, 1, 3, 5, 7, 2, 8, 6]
EP = [4, 1, 2, 3, 2, 3, 4, 1]

# Permutations for S-boxes
P4 = [2, 4, 3, 1]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]

# S-Box Tables
S0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]
S1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

# Circular left shift function
def left_shift(bits, n):
    return bits[n:] + bits[:n]

# Initial permutation
def initial_permutation(plaintext):
    return [plaintext[i - 1] for i in IP]

# Inverse initial permutation
def inverse_initial_permutation(ciphertext):
    return [ciphertext[i - 1] for i in IP_INV]

# Expand and permute function
def expand_permute(bits):
    return [bits[i - 1] for i in EP]

# Permute 4 function
def permute4(bits):
    return [bits[i - 1] for i in P4]

# Permute 8 function
def permute8(bits):
    return [bits[i - 1] for i in P8]

# Permute 10 function
def permute10(bits):
    return [bits[i - 1] for i in P10]

# S-Box lookup function
def sbox_lookup(bits, sbox):
    row = int(''.join([str(bits[0]), str(bits[3])]), 2)
    col = int(''.join([str(bits[1]), str(bits[2])]), 2)
    return format(sbox[row][col], '02b')

# F function
def f_function(bits, key):
    expanded = expand_permute(bits)
    xor_result = [int(expanded[i]) ^ int(key[i]) for i in range(len(expanded))]
    sbox0_input = xor_result[:4]
    sbox1_input = xor_result[4:]
    sbox0_output = sbox_lookup(sbox0_input, S0)
    sbox1_output = sbox_lookup(sbox1_input, S1)
    sbox_output = sbox0_output + sbox1_output
    return permute4([int(sbox_output[i]) for i in range(len(sbox_output))])

# Generate round keys
def generate_round_keys(key):
    key = permute10(key)
    key = left_shift(key[:5], 1) + left_shift(key[5:], 1)
    round_key1 = permute8(key)
    key = left_shift(key[:5], 2) + left_shift(key[5:], 2)
    round_key2 = permute8(key)
    return round_key1, round_key2

# Encrypt function
def encrypt(plaintext, key):
    plaintext = initial_permutation(plaintext)
    round_key1, round_key2 = generate_round_keys(key)
    # Round 1
    plaintext_left = plaintext[:4]
    plaintext_right = plaintext[4:]
    f_output = f_function(plaintext_right, round_key1)
    xor_result = [int(plaintext_left[i]) ^ int(f_output[i]) for i in range(len(plaintext_left))]
    # Round 2
    plaintext_left = plaintext_right
    plaintext_right = xor_result
    f_output = f_function(plaintext_right, round_key2)
    xor_result = [int(plaintext_left[i]) ^ int(f_output[i]) for i in range(len(plaintext_left))]
    ciphertext = xor_result + plaintext_right
    return inverse_initial_permutation(ciphertext)

# Decrypt function
def decrypt(ciphertext, key):
    ciphertext = initial_permutation(ciphertext)
    round_key1, round_key2 = generate_round_keys(key)
    # Round 1
    ciphertext_left = ciphertext[:4]
    ciphertext_right = ciphertext[4:]
    f_output = f_function(ciphertext_right, round_key2)
    xor_result = [int(ciphertext_left[i]) ^ int(f_output[i]) for i in range(len(ciphertext_left))]
    # Round 2
    ciphertext_left = ciphertext_right
    ciphertext_right = xor_result
    f_output = f_function(ciphertext_right, round_key1)
    xor_result = [int(ciphertext_left[i]) ^ int(f_output[i]) for i in range(len(ciphertext_left))]
    plaintext = xor_result + ciphertext_left
    return inverse_initial_permutation(plaintext)

# Example usage
plaintext = '10100010'
key = '1010000010'

encrypted = encrypt(plaintext, key)
print("Encrypted:", ''.join(map(str, encrypted)))

decrypted = decrypt(encrypted, key)
print("Decrypted:", ''.join(map(str, decrypted)))

Encrypted: 01100011
Decrypted: 10100011
